In [ ]:
pip install wandb

In [ ]:
!wandb login

wandb: Currently logged in as: tandat88963820 (tandat88963820-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np
import pandas as pd

In [ ]:

data = np.genfromtxt('advertising.csv',
                     delimiter=',',
                     skip_header=1).tolist()

TVs = []
radios = []
newspapers = []
sales = []
N=len(data)
for sample in data:
    TVs.append(sample[0])
    radios.append(sample[1])
    newspapers.append(sample[2])
    sales.append(sample[3])

In [ ]:
def predict(x1, x2, x3, w1, w2, w3, b):
    return w1*x1 + w2*x2 + w3*x3 + b

def compute_loss(y_hat, y):
    return (y_hat - y)**2

def compute_derivatives(x1, x2, x3, y_hat, y):
    dl_dw1 = 2*(y_hat - y)*x1
    dl_dw2 = 2*(y_hat - y)*x2
    dl_dw3 = 2*(y_hat - y)*x3
    dl_db  = 2*(y_hat - y)
    return dl_dw1, dl_dw2, dl_dw3, dl_db
def update(w1, w2, w3, b, lr, dl_dw1, dl_dw2, dl_dw3, dl_db):
    w1 = w1 - lr*dl_dw1
    w2 = w2 - lr*dl_dw2
    w3 = w3 - lr*dl_dw3
    b  = b  - lr*dl_db
    return w1, w2, w3, b


In [ ]:
import wandb
import pandas as pd
w1=0
w2=0
w3=0
b=0
lr=0.00001
epochs = 100
wandb.init(
    project="demo_linear_regression",
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)
df = pd.DataFrame(data, columns=["TVs", "Radio", "Newspaper", "Sales"])
wandb.run.log({"data": wandb.Table(dataframe=df)})
losses = []
for epoch in range(epochs):
    for i in range(N):
        x1, x2, x3, y = TVs[i], radios[i], newspapers[i], sales[i]

        y_hat = predict(x1, x2, x3, w1, w2, w3, b)
        loss = compute_loss(y_hat, y)
        losses.append(loss)

        dl_dw1, dl_dw2, dl_dw3, dl_db = compute_derivatives(x1, x2, x3, y_hat, y)
        w1, w2, w3, b = update(w1, w2, w3, b, lr, dl_dw1, dl_dw2, dl_dw3, dl_db)
        wandb.log({"loss": loss})
wandb.finish()


loss,▄▃▁▁▂▄▁▅▁▁▃▁▅▁▁▂▅█▃▂▃▁▂▁▄▂▁▂▂▃▁▁▁▆▄▁▁▁▃▂
loss,29.20745
